In [2]:
import numpy as np
import matplotlib.pyplot as plt
import iDEA
import ipywidgets as widgets
from IPython.display import display
import pickle
import time
from pathlib import Path
import collections_iDEA
import collections_iDEA.multiplets as mm
import os
import scipy.special as sp

# Initialize the system
points = 231
l = 10
x = np.linspace(-l, l, points)
dx = x[1] - x[0]
omega = 0.3275
v_ext = 0.5 * omega**2 * (x**2)
v_int = np.zeros([len(x), len(x)])
qho_single = iDEA.system.System(x, v_ext, v_int, electrons="u")
qho_double = iDEA.system.System(x, v_ext, v_int, electrons="uu")

states_small_int = {}
states_zero_int = {}
occupied_info = []

directory = Path("states")

# for i in range(0, 5):
#     file_path = directory / f"qho_small_int_state_{i}_1e-11.pkl"
#     try:
#         with file_path.open("rb") as file:
#             states_small_int[i] = pickle.load(file)
#     except FileNotFoundError:
#         print(f"File state_{i}.pkl not found.")
#     except Exception as e:
#         print(f"An error occurred while loading state_{i}.pkl: {e}")

max_k = 7

iDEA_collection = collections_iDEA.CollectionOfStates(max_k)
energy_array = np.zeros(max_k)

for i in range(max_k):
    file_path = directory / f"qho_double_state_{i}_231_points.pkl"
    try:
        with file_path.open("rb") as file:
            state = pickle.load(file)
            iDEA_collection.add_state_class(state)
            energy_array[i] = state.energy
    except FileNotFoundError:
        print(f"File state_{i}.pkl not found.")
    except Exception as e:
        print(f"An error occurred while loading state_{i}.pkl: {e}")

iDEA_collection.add_energies(energy_array)




def qho_energy(index):
    return omega*(index + 0.5)
index_list = collections_iDEA.CollectionOfStates(7)
mm.apply_energy_method(qho_energy, qho_double, 7, states=index_list, second_mask=False)
index_list.calculate_multiplets()

for i in range(len(index_list.up_indices)):

    print(f"Analytic State {i}, First Index: {index_list.up_indices[i]}, Second Index: {index_list.down_indices[i]}, Energy: {index_list.energies[i]}")

print(f"Multiplets: {index_list.multiplets}")

iDEA_collection.calculate_multiplets()
print(f"Multiplets: {iDEA_collection.multiplets}")
print(index_list.check_expected_num_of_multiplets(iDEA_collection.multiplets, iDEA_collection.multiplet_energies))

File indices_up_to_20.pkl exists
Elapsed Time = 0.0049981150004896335
Analytic State 0, First Index: 0, Second Index: 1, Energy: 0.655
Analytic State 1, First Index: 1, Second Index: 0, Energy: 0.655
Analytic State 2, First Index: 0, Second Index: 2, Energy: 0.9825000000000002
Analytic State 3, First Index: 2, Second Index: 0, Energy: 0.9825000000000002
Analytic State 4, First Index: 0, Second Index: 3, Energy: 1.31
Analytic State 5, First Index: 1, Second Index: 2, Energy: 1.31
Analytic State 6, First Index: 2, Second Index: 1, Energy: 1.31
Multiplets: [[0, 1], [2, 3]]
Multiplets: [[4, 5]]
{1: ('Got: [4, 5] with energy: 1.310000000307644', 'Expected: [0, 1]', 'Expected energy: 0.655')}


In [ ]:
# for key, value in states_small_int.items():
#     print(f"iDEA State {key}, Energy = {value.energy}")

for i in range(len(iDEA_collection.state_classes)):
    print(f"iDEA State {i}, Energy = {iDEA_collection.energies[i]}")

In [ ]:
analytic_prob_densities = []

def wave_function(index, omega, x):
    exponent = -(omega * (x**2)) / (2)
    t = np.sqrt(1 / ((2**index) * sp.factorial(index))) * pow((omega/np.pi), 0.25) * np.exp(exponent) * sp.eval_hermite(index, np.sqrt(omega)*x)

    return t

# ind = [[0, 1], [0, 2], [1, 2], [2, 1], [0, 3]]

for i in range(7):
    index_k = index_list.up_indices[i]
    index_j = index_list.down_indices[i]
    # index_k = ind[i][0]
    # index_j = ind[i][1]
    print(f"State {i}, {index_k} | {index_j}")

    pd_analytic_1 = (np.abs(wave_function(index_k, omega, x)))**2 + (np.abs(wave_function(index_j, omega, x)))**2
    analytic_prob_densities.append(pd_analytic_1)

In [ ]:
collections_iDEA.analysis.states_density_comparison(iDEA_collection, qho_double, analytic_prob_densities)

In [ ]:
analytic_prob_densities = []

def wave_function(index, omega, x):
    exponent = -(omega * (x**2)) / (2)
    t = np.sqrt(1 / ((2**index) * sp.factorial(index))) * pow((omega/np.pi), 0.25) * np.exp(exponent) * sp.eval_hermite(index, np.sqrt(omega)*x)

    return t

ind = [[0, 1], [0, 2], [1, 2], [2, 1], [0, 3], [3, 0], [1, 3]]

for i in range(7):
    # index_k = index_list.up_indices[i]
    # index_j = index_list.down_indices[i]
    index_k = ind[i][0]
    index_j = ind[i][1]
    print(f"State {i}, {index_k} | {index_j}")

    pd_analytic_1 = (np.abs(wave_function(index_k, omega, x)))**2 + (np.abs(wave_function(index_j, omega, x)))**2
    analytic_prob_densities.append(pd_analytic_1)

In [ ]:
collections_iDEA.analysis.states_density_comparison(iDEA_collection, qho_double, analytic_prob_densities)

In [ ]:
diff = []
n = iDEA.observables.density(qho_double, iDEA_collection.state_classes[6])        


for i in range(len(n)):
        diff.append(abs(n[i] - analytic_prob_densities[6][i]))



plt.plot(x, diff)

# int = 0

In [ ]:
max_k = 7

iDEA_collection = collections_iDEA.CollectionOfStates(max_k)
energy_array = np.zeros(max_k)

for i in range(max_k):
    file_path = directory / f"qho__zero_double_state_{i}_231_points.pkl"
    try:
        with file_path.open("rb") as file:
            state = pickle.load(file)
            iDEA_collection.add_state_class(state)
            energy_array[i] = state.energy
    except FileNotFoundError:
        print(f"File state_{i}.pkl not found.")
    except Exception as e:
        print(f"An error occurred while loading state_{i}.pkl: {e}")

iDEA_collection.add_energies(energy_array)

In [ ]:
def qho_energy(index):
    return omega*(index + 0.5)

index_list = mm.apply_energy_method(qho_energy, qho_double, 9, second_mask=False)
index_list.calculate_multiplets()

for i in range(len(index_list.up_indices)):

    print(f"Analytic State {i}, First Index: {index_list.up_indices[i]}, Second Index: {index_list.down_indices[i]}, Energy: {index_list.energies[i]}")

print(f"Multiplets: {index_list.multiplets}")

In [ ]:
for i in range(len(iDEA_collection.state_classes)):
    print(f"iDEA State {i}, Energy = {iDEA_collection.energies[i]}")

In [ ]:
analytic_prob_densities = []

def wave_function(index, omega, x):
    exponent = -(omega * (x**2)) / (2)
    t = np.sqrt(1 / ((2**index) * sp.factorial(index))) * pow((omega/np.pi), 0.25) * np.exp(exponent) * sp.eval_hermite(index, np.sqrt(omega)*x)

    return t

# ind = [[0, 1], [0, 2], [1, 2], [2, 1], [0, 3]]

for i in range(7):
    index_k = index_list.up_indices[i]
    index_j = index_list.down_indices[i]
    # index_k = ind[i][0]
    # index_j = ind[i][1]
    print(f"State {i}, {index_k} | {index_j}")

    pd_analytic_1 = (np.abs(wave_function(index_k, omega, x)))**2 + (np.abs(wave_function(index_j, omega, x)))**2
    analytic_prob_densities.append(pd_analytic_1)

In [ ]:
collections_iDEA.analysis.states_density_comparison(iDEA_collection, qho_double, analytic_prob_densities)

In [ ]:
analytic_prob_densities = []

def wave_function(index, omega, x):
    exponent = -(omega * (x**2)) / (2)
    t = np.sqrt(1 / ((2**index) * sp.factorial(index))) * pow((omega/np.pi), 0.25) * np.exp(exponent) * sp.eval_hermite(index, np.sqrt(omega)*x)

    return t

ind = [[0, 1], [0, 2], [1, 2], [2, 1], [0, 3], [3, 0], [1, 3]]

for i in range(7):
    # index_k = index_list.up_indices[i]
    # index_j = index_list.down_indices[i]
    index_k = ind[i][0]
    index_j = ind[i][1]
    print(f"State {i}, {index_k} | {index_j}")

    pd_analytic_1 = (np.abs(wave_function(index_k, omega, x)))**2 + (np.abs(wave_function(index_j, omega, x)))**2
    analytic_prob_densities.append(pd_analytic_1)

In [ ]:
collections_iDEA.analysis.states_density_comparison(iDEA_collection, qho_double, analytic_prob_densities)

In [ ]:
diff = []
n = iDEA.observables.density(qho_double, iDEA_collection.state_classes[6])        


for i in range(len(n)):
        diff.append(abs(n[i] - analytic_prob_densities[6][i]))



plt.plot(x, diff)